In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import nni
import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Base directory path
base_path = "G:\ABCD\data"

# File paths using the base path
file1_1_path = f"{base_path}/mri_y_rsfmr_cor_gp_aseg.csv"
file1_2_path = f"{base_path}/mri_y_rsfmr_cor_gp_gp.csv"
file1_3_path = f"{base_path}/mri_y_rsfmr_var_aseg.csv"
file1_4_path = f"{base_path}/mri_y_rsfmr_var_dsk.csv"
file1_5_path = f"{base_path}/mri_y_rsfmr_var_dst.csv"
file1_6_path = f"{base_path}/mri_y_rsfmr_var_gp.csv"


file2_path = f"{base_path}/mh_p_cbcl.csv"


file1_1 = pd.read_csv(file1_1_path)
file1_2 = pd.read_csv(file1_2_path)
file1_3 = pd.read_csv(file1_3_path)
file1_4 = pd.read_csv(file1_4_path)
file1_5 = pd.read_csv(file1_5_path)
file1_6 = pd.read_csv(file1_6_path)

dfs = [file1_1, file1_2, file1_3, file1_4, file1_5, file1_6]

dfs = [df[df['eventname'] == 'baseline_year_1_arm_1'].drop(columns=['eventname']) for df in dfs]

# 分别将筛选后的数据框赋值回原来的变量
file1_1, file1_2, file1_3, file1_4, file1_5, file1_6 = dfs


file1 = pd.merge(file1_1, file1_2, on='src_subject_id', how='inner')
file1 = pd.merge(file1, file1_3, on='src_subject_id', how='inner')
file1 = pd.merge(file1, file1_4, on='src_subject_id', how='inner')
file1 = pd.merge(file1, file1_5, on='src_subject_id', how='inner')
file1 = pd.merge(file1, file1_6, on='src_subject_id', how='inner')

file1 = file1.loc[:, file1.isnull().mean() < .5]


file2 = pd.read_csv(file2_path)

file2_baseline_filtered = file2[file2['eventname'] == 'baseline_year_1_arm_1']

#drop score
file2_drop = file2_baseline_filtered.loc[:, ~file2_baseline_filtered.columns.str.endswith(('_r', '_t', '_m', '_nm', '_nm_2', '___1'))]
#drop eventname
file2_drop = file2_drop.drop(columns=['eventname'])
file2_final_cleaned = file2_drop.dropna(axis=0)



C:\Users\Makka Papa\AppData\Local\Temp\ipykernel_6416\38619458.py:51: DtypeWarning: Columns (124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200) have mixed types. Specify dtype option on import or set low_memory=False.
  file2 = pd.read_csv(file2_path)


In [2]:
file2_baseline_filtered

,src_subject_id,eventname,cbcl_select_language___1,cbcl_q01_p,cbcl_q02_p,cbcl_q03_p,cbcl_q04_p,cbcl_q05_p,cbcl_q06_p,cbcl_q07_p,...,cbcl_scr_07_sct_nm,cbcl_scr_07_ocd_r,cbcl_scr_07_ocd_t,cbcl_scr_07_ocd_m,cbcl_scr_07_ocd_nm,cbcl_scr_07_stress_r,cbcl_scr_07_stress_t,cbcl_scr_07_stress_m,cbcl_scr_07_stress_nm,cbcl_scr_07_stress_nm_2
0,NDAR_INV003RTV85,baseline_year_1_arm_1,0,0,0,0,0,0,0,0,...,0.0,0.0,50.0,NaN,0.0,0.0,50.0,NaN,0.0,NaN
4,NDAR_INV005V6D2C,baseline_year_1_arm_1,1,0,0,0,0,0,0,0,...,0.0,1.0,51.0,NaN,0.0,0.0,50.0,NaN,0.0,NaN
8,NDAR_INV007W6H7B,baseline_year_1_arm_1,0,0,0,1,0,0,0,1,...,0.0,3.0,60.0,NaN,0.0,4.0,54.0,NaN,0.0,NaN
11,NDAR_INV00BD7VDC,baseline_year_1_arm_1,0,0,0,1,1,0,0,1,...,0.0,4.0,64.0,NaN,0.0,4.0,54.0,NaN,0.0,NaN
14,NDAR_INV00CY2MDM,baseline_year_1_arm_1,0,1,0,2,1,0,0,1,...,0.0,0.0,50.0,NaN,0.0,7.0,62.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48715,NDAR_INVZZNX6W2P,baseline_year_1_arm_1,0,0,0,1,0,1,0,0,...,0.0,0.0,50.0,NaN,0.0,3.0,51.0,NaN,0.0,NaN
48720,NDAR_INVZZPKBDAC,baseline_year_1_arm_1,0,0,0,0,0,0,0,0,...,0.0,0.0,50.0,NaN,0.0,1.0,50.0,NaN,0.0,NaN
48725,NDAR_INVZZZ2ALR6,baseline_year_1_arm_1,0,0,0,1,0,0,0,0,...,0.0,1.0,51.0,NaN,0.0,1.0,50.0,NaN,0.0,NaN
48730,NDAR_INVZZZNB0XC,baseline_year_1_arm_1,0,1,0,0,1,0,0,0,...,0.0,1.0,51.0,NaN,0.0,1.0,50.0,NaN,0.0,NaN


In [21]:
merged_data = pd.merge(file1, file2_final_cleaned, on='src_subject_id', how='inner')
merged_data = merged_data.dropna(axis=0)

In [24]:
# file1.to_csv('features.csv')
# file2_final_cleaned.to_csv('labels.csv')
merged_data.to_csv('merged.csv')

# All years

In [8]:
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import nni
import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Base directory path
base_path = "G:\ABCD\data"

# File paths using the base path
file1_1_path = f"{base_path}/mri_y_rsfmr_cor_gp_aseg.csv"
file1_2_path = f"{base_path}/mri_y_rsfmr_cor_gp_gp.csv"
file1_3_path = f"{base_path}/mri_y_rsfmr_var_aseg.csv"
file1_4_path = f"{base_path}/mri_y_rsfmr_var_dsk.csv"
file1_5_path = f"{base_path}/mri_y_rsfmr_var_dst.csv"
file1_6_path = f"{base_path}/mri_y_rsfmr_var_gp.csv"


file2_path = f"{base_path}/mh_p_cbcl.csv"


file1_1 = pd.read_csv(file1_1_path)
file1_2 = pd.read_csv(file1_2_path)
file1_3 = pd.read_csv(file1_3_path)
file1_4 = pd.read_csv(file1_4_path)
file1_5 = pd.read_csv(file1_5_path)
file1_6 = pd.read_csv(file1_6_path)

dfs = [file1_1, file1_2, file1_3, file1_4, file1_5, file1_6]

# dfs = [df[df['eventname'] == 'baseline_year_1_arm_1'].drop(columns=['eventname']) for df in dfs]

# 分别将筛选后的数据框赋值回原来的变量
file1_1, file1_2, file1_3, file1_4, file1_5, file1_6 = dfs

selected_columns = ['src_subject_id', 'eventname']
file1 = pd.merge(file1_1, file1_2, on= selected_columns, how='inner')
file1 = pd.merge(file1, file1_3, on= selected_columns, how='inner')
file1 = pd.merge(file1, file1_4, on=selected_columns, how='inner')
file1 = pd.merge(file1, file1_5, on=selected_columns, how='inner')
file1 = pd.merge(file1, file1_6, on=selected_columns, how='inner')


In [9]:

file1 = file1.loc[:, file1.isnull().mean() < .5]


file2 = pd.read_csv(file2_path)

# file2_baseline_filtered = file2[file2['eventname'] == 'baseline_year_1_arm_1']
file2_baseline_filtered = file2

#drop score
file2_drop = file2_baseline_filtered.loc[:, ~file2_baseline_filtered.columns.str.endswith(('_r', '_t', '_m', '_nm', '_nm_2', '___1'))]
#drop eventname
# file2_drop = file2_drop.drop(columns=['eventname'])
file2_final_cleaned = file2_drop.dropna(axis=0)



C:\Users\Makka Papa\AppData\Local\Temp\ipykernel_4008\4158922468.py:4: DtypeWarning: Columns (124,128,132,136,140,144,148,152,156,160,164,168,172,176,180,184,188,192,196,200) have mixed types. Specify dtype option on import or set low_memory=False.
  file2 = pd.read_csv(file2_path)


In [12]:
file2_final_cleaned.to_csv('all_year_labels.csv',index=False)